In [2]:
import requests

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_paper_authors = pd.read_csv('../data/author benchmarks - paper_authors.csv')

In [6]:
df_paper_authors.doi.notna().sum()

2651

In [7]:
dois = df_paper_authors['doi'].dropna().unique()

endpoint = "works"
size = 50
loop_index = 0
works = []
for list_index in range(0, len(dois), size):
    subset = dois[list_index:list_index+size]
    pipe_separated_ids = "|".join(subset)
    r = requests.get(f"https://api.openalex.org/{endpoint}?filter=doi:{pipe_separated_ids}&per-page={size}")
    results = r.json()['results']
    works.extend(results)
    loop_index += 1
print(f"collected {len(works)} works using {loop_index} api calls")

collected 2359 works using 48 api calls


In [8]:
data = []
for work in works:
    doi = work['doi'].replace('https://doi.org/', '')
    authors = []
    for position, authorship in enumerate(work['authorships']):
        author_name = authorship['author']['display_name']
        author_id = authorship['author']['id'].replace('https://openalex.org/', '')
        authors.append( (position, author_name, author_id) )
    data.append({
        'doi': doi,
        'authors': authors,
    })
openalex_author_info = pd.DataFrame(data)

In [9]:
df_paper_authors = df_paper_authors.merge(openalex_author_info, how='left', on='doi')

In [11]:
df_paper_authors.to_csv('../data/scopus_paper_authors_with_openalex_author_info.csv')